# Introduction to bokeh

## Top 5 fun and entertaining plots
---

<img src='./images/logos.3.600.wide.png' height='250' width='300' style="float:right">

Next, we will take a look at five sample plots include many diverse examples of how bokeh can be used for awesome data visualizations.
   0. **scatterplot**
   1. **histogram** with pdf and cdf curves
   2. graph with **interactive sliders** to set parameters
   3. **visualization tools**: pan, zoom, hover, crosshairs, lasso select, box select, poly select, save, undo, redo, tap
   4. **geographical map**
 

This material is based largely upon the sample graphs shown here:
http://bokeh.pydata.org/en/latest/docs/gallery.html
 

# Sample data for your amusement is available:



Bokeh comes preinstalled with various types of sample data: look in the site-packages folder found in your virtual environment folder...

`<path_to_where_you_installed_miniconda>/miniconda3/envs/bokeh_tut/lib/python3.5/site-packages/bokeh/sampledata/`

Bokeh provides ready access to some of the smaller datasets via import, as we will see.

There are other much larger datasets available as well, that we will be using and will get later, using: 

`bokeh.sampledata.download()`


# Simple scatterplot
---

<img src='./images/220px-Iris_versicolor_3.jpg' height='250' width='300' style="float:right">

In [ ]:
# Let's start by importing some of the data

from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.iris import flowers

# When flowers is imported, it comes in as a pandas DataFrame
# DataFrames have a function called sample, that will let us see a representative
#     sample of all the data... let's look at 10 rows.

print(type(flowers))
print(len(flowers))
flowers.sample(10)


In [ ]:
# Next, we will prep some enrichments that we can apply to our flower data
# We want to map the species of iris to a color so we create a dict.

colormap = {'setosa': 'red',
            'versicolor': 'green',
            'virginica': 'blue'}

# Then we create a list of colors... one color for each row in the flowers DataFrame.
# Based on the species in the species column, we insert the appropriate color
# into our list.

colors = [colormap[x] for x in flowers['species']]

# To see the results of our work, let's look at a sampling of the colors list.

print(len(colors))
colors[::10]

In [ ]:

p = figure(title = "Iris Morphology")

p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

p.circle(flowers["petal_length"],      # x values
         flowers["petal_width"],       # y values
         color=colors, 
         fill_alpha=0.2, 
         size=10)

In [ ]:
output_notebook()
show(p)

# http://bokeh.pydata.org/en/latest/docs/gallery/iris.html

# Workflow
---

As we look at the remaining examples, and as you create your own, a generic, but useful workflow is listed below. Following a workflow such as this, helps you to focus on one task at a time and create robust and easy to understand scripts.

1. Import libraries
1. Generate a figure
1. Create or load data and enrichments
1. Generate glyphs
1. Add attributes, annotations, interactions
1. Create outputs

**NOTES**:

* You may not need some steps in the above workflow, especially for barebones charts.
* Conversely, your charts may be very sophisticated and you may need to expand on the above workflow. 
* This is intended as a starting point. 
* Your mileage may vary (YMMV)!


# Histogram: with pdf and cdf curves
---

In [ ]:
# IMPORTS ---------------------------------------

import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook

In [ ]:
# GENERATE FIGURE -----------------------------------------

h = figure(title="Normal Distribution (μ=0, σ=0.5)",
            tools="save",
            background_fill_color="whitesmoke")

# Notice this creates a figure() object

h

```
h?
```

versus

```
h??
```

In [ ]:
# CREATE OR LOAD DATA/ENRICHMENTS -------------------------

# set the midpoint and spread of the data
mu, sigma = 0, 0.5

# draw a 1000 random samples from a normal distribution 
measured = np.random.normal(mu, sigma, 1000)

# compute the histogram of a set of data.
#     hist is an array of the y-values of the histogram points
#     edges is an array of the 'sides' of the rectangles as 
#     defined across the x-axis
hist, edges = np.histogram(measured, density=True, bins=50)

# create an array of x-values: 1000 values between -2 and 2
x = np.linspace(-2, 2, 1000)

# calculate the points for a pdf and a cdf based on:
#     * mu
#     * sigma
#     * the values of x
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

print('MEASURED: ', measured[:10])
print()
print('HIST: ', len(hist), hist[:10])
print()
print('EDGES: ', len(edges), edges[:10])
print()
print('-' * 60, end='\n\n')
print('PDF: ', pdf[:10])
print()
print('CDF: ', cdf[:10])


# Experience Points
---

1. In the code cell below, use `?` to explore the `quad()` function assocated with the figure `h`
1. What is the default fill_color?
1. What does the characteristic `alpha` do?
1. How does `alpha` contrast with `line_alpha`?

In [ ]:
# Let's look into the quad() method of our figure:



In [ ]:
# GENERATE GLYPHS -----------------------------------------

h.quad(top=hist,
       bottom=0,
       left=edges[:-1],
       right=edges[1:],
       fill_color='yellow', alpha=0.9, line_color="orange")


In [ ]:
# GENERATE GLYPHS (cont.)-----------------------------------

h.line(x, pdf, line_color="black", line_width=3, alpha=0.7, legend="PDF")
h.line(x, cdf, line_color="red", line_width=2, alpha=0.7, legend="CDF")

In [ ]:
# ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS ---------------

h.legend.location = "top_left"
h.xaxis.axis_label = 'x'
h.yaxis.axis_label = 'Pr(x)'

In [ ]:
# CREATE OUTPUTS ------------------------------------------
output_notebook()
show(h)

# http://bokeh.pydata.org/en/latest/docs/gallery/histogram.html

# Using interactive sliders
---

In [ ]:
# IMPORTS -------------------------------------------------

import numpy as np

from bokeh.layouts import row, widgetbox

from bokeh.models import CustomJS, Slider

from bokeh.plotting import figure, output_notebook, show, ColumnDataSource

# Experience Points
---

# Try to identify the purpose of these two functions:

1. CustomJS
1. Slider

In [ ]:
# GENERATE FIGURE -----------------------------------------
plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

In [ ]:
# CREATE OR LOAD DATA/ENRICHMENTS -------------------------
x = np.linspace(0, 10, 500)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

In [ ]:
# GENERATE GLYPHS -----------------------------------------
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

In [ ]:
# ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS ---------------
# I am not a javascript guru.
# ... you are on your own in terms of making javascript


callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var A = amp.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = A*Math.sin(x[i]);
    }
    source.trigger('change');
    """)

amp_slider = Slider(start=0.1, end=10, value=1, step=.1,
                    title="Amplitude",
                    callback=callback)

callback.args["amp"] = amp_slider

layout = row(plot,
             widgetbox(amp_slider))

In [ ]:
# CREATE OUTPUTS ------------------------------------------
output_notebook()
show(layout)

# http://bokeh.pydata.org/en/latest/docs/gallery/slider.html

# Using visualization tools
---

In [ ]:
# IMPORTS -------------------------------------------------

import numpy as np
from bokeh.plotting import figure, show, output_notebook

In [ ]:
# CREATE OR LOAD DATA/ENRICHMENTS -------------------------

number = 4000
x = np.random.random(size=number) * 100     # random yields [0.0, 1.0) OR 0.0 <= x < 1.0
y = np.random.random(size=number) * 100

radii = np.random.random(size=number) * 1.5

# The following string formatting produces a hex number:
# %02x yields a zero padded hex number when you provide an integer
#     "#%02x" % (int(10)) would yield #0a
#     "#%02x%02x%02x" % (int(10), int(11), 150) would yield #0a0b96

colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

In [ ]:
# GENERATE FIGURE -----------------------------------------

TOOLS="crosshair,hover,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select"

p = figure(tools=TOOLS)

# GENERATE GLYPHS -----------------------------------------

p.scatter(x, y, radius=radii,
          fill_color=colors, fill_alpha=0.6,
          line_color=None)


# CREATE OUTPUTS ------------------------------------------

output_notebook()
show(p)  # open a browser

# http://bokeh.pydata.org/en/latest/docs/gallery/color_scatter.html

# Experience Points
---

Edit the code cell above to experiment with tool settings:

1. Add an **additional** `pan` tool to the TOOLS string and execute the code
   * Consider the warning you get
   * Look at the toolbar, anyway
   * Remove the extra `pan` tool
1. Add the tool: `poly_select` to the TOOLS string and execute the code
1. Add the tool: `lasso_select` to the TOOLS string and execute the code

# Geographical map
---

In [ ]:
# For larger datasets, we will need to download the data 

# The data will be stored in the following location:
# $home_directory/.bokeh/data

import bokeh
bokeh.sampledata.download()

In [ ]:
# IMPORTS -------------------------------------------------

from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)

from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure, output_notebook


from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment


In [ ]:
# GENERATE FIGURE -----------------------------------------

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

texas = figure(title="Texas Unemployment, 2009", 
               tools=TOOLS, 
               x_axis_location=None,
               y_axis_location=None)

texas.grid.grid_line_color = None


In [ ]:
# Let's understand the data:
# Looking at just the first 20 keys from the counties data, we see that they are 
# tuples with the first number equal to 48
# the second number is an odd number

print(sorted(counties.keys())[:20])       
print()

# Looking at just one set of data for the county 
# coded with tuple 48, 3, we see that the data is a state, some lats and longs and some names.

print(counties[(48,3)])

In [ ]:
# Looking at the data for unemployment:
# Each key is a tuple, some of which match the tuples for the counties
# Each tuple is paired with an unemployment value.

for key, value in sorted(unemployment.items()):
    if key[0] == 48:
        print('{}: {}'.format(key, value))


In [ ]:
# CREATE OR LOAD DATA/ENRICHMENTS -------------------------

palette.reverse()

counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]


# LogColorMapper will map numbers in a range [low, high] against a
# sequence of colors (i.e. a palette) on a natural logarithm scale.

color_mapper = LogColorMapper(palette=palette)

# ColumnDataSource() maps the names of columns to sequences or arrays,
# often using a dictionary OR DataFrame. Here we map all the values 
# of county_xs to a variable called x, etc.

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
))


In [ ]:
# GENERATE GLYPHS -----------------------------------------

# Here, we are referring to the 'x' and 'y' data found in the 
# ColumnDataSource we just created.
# Note that we use the string 'x' NOT x

texas.patches('x', 'y', source=source,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)


In [ ]:
# ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS ---------------

hover = texas.select_one(HoverTool)

hover.point_policy = "follow_mouse"

hover.tooltips = [("Name", "@name"),
                  ("Unemployment rate)", "@rate%"),
                  ("(Long, Lat)", "($x, $y)"),]


In [ ]:
# CREATE OUTPUTS ------------------------------------------

output_notebook()
show(texas)

# http://bokeh.pydata.org/en/latest/docs/gallery/texas.html

# Experience Points
---

1. On the Internets, research the syntax for generating tool tips.


## Navigation
---

| Previous | Up | Next |
|:-----|:-----:|-----:|
| <<< [First Graph](./first_graph.ipynb) | [Table of Contents](./README.md) | [What Went Wrong](./what_went_wrong.ipynb) >>> |